## Model Building

##### Predicting wethere the water quality is safe for drinking or not by building ML models such as Logistic regression, Support vector Machine(SVM) and Decision tree along with hyperparameter tuning to predict the chance of diabetes. Using Recall as a performance metric to judge our models.

##### Choosing Recall as a performance metric helps to prioritise FALSE NEGATIVES where the model predicts the water is safe to drink but in reality it isn't, thus, creating a higher chance of risking lives.

### Lodaing the libraries 

In [47]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier 
from matplotlib import pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import RandomOverSampler
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeRegressor
import warnings
warnings.filterwarnings("ignore")


### Loading the processed training and test datasets

In [48]:
X_train = pd.read_csv('/Users/shambhavimishra/Downloads/X_train.csv') 
y_train = pd.read_csv('/Users/shambhavimishra/Downloads/y_train.csv') 
X_test = pd.read_csv('/Users/shambhavimishra/Downloads/X_test.csv') 
y_test = pd.read_csv('/Users/shambhavimishra/Downloads/y_test.csv') 

In [49]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(14847, 19)
(14847, 1)
(2400, 19)
(2400, 1)


### Building a dataframe to store our models performance metrics

In [50]:
performance = pd.DataFrame({"model": [], "Accuracy": [], "Precision": [], "Recall": [], "F1": []})

### Implementing Logistic Regression model with hyperparameters

##### Using Randomized search to find the best parameters

In [51]:
param_grid = {'penalty': ['l1', 'l2'], 
              'C': [0.001, 0.01, 0.1, 1, 10, 100],
              'solver': ['liblinear', 'saga'],
              'l1_ratio': [0.25, 0.5, 0.75],
             'max_iter': np.arange(800, 1200)
             }

# Perform Randomized Search CV to find the best hyperparameters
best_log_reg = RandomizedSearchCV(estimator=LogisticRegression(random_state=0, solver='saga'),
                                      scoring='recall', 
                                      param_distributions=param_grid, 
                                      cv=5, 
                                      verbose=0, 
                                      return_train_score=True, 
                                      n_iter=30, 
                                      n_jobs=-1)
best_log_reg.fit(X_train, y_train)

# Print the best parameters found through Randomized Search CV
print(f"Best parameters found through Randomized Search CV: {best_log_reg.best_params_}")



/Users/shambhavimishra/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/Users/shambhavimishra/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/Users/shambhavimishra/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1165: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l2)
  warnings.warn(
/Users/shambhavimishra/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/shambhavimishra/opt/anaconda3/lib/python3.9/site-packages/sklea

Best parameters found through Randomized Search CV: {'solver': 'saga', 'penalty': 'l2', 'max_iter': 961, 'l1_ratio': 0.25, 'C': 0.01}


##### Performing GridSearch over a close range of parameters that we got from Randomized search to find the best parameters

In [54]:
# Define the parameter grid for Grid Search CV
param_grid = { 
    'solver': ['saga'],
    'penalty': ['l2'],
    'C': [0.1, 1, 10],
    'max_iter': np.arange(750,950)
}

# Perform Grid Search CV with the best parameters from Randomized Search CV
grid_log_reg = GridSearchCV(estimator=LogisticRegression(random_state=0, solver=best_lregression.best_params_['solver']),
                                param_grid=param_grid,
                                scoring='recall',
                                cv=5,
                                n_jobs=-1)
grid_log_reg.fit(X_train, y_train)

# Print the best parameters found through Grid Search CV
print(f"Best parameters found through Grid Search CV: {grid_log_reg.best_params_}")

AttributeError: 'RandomizedSearchCV' object has no attribute 'best_params_'

##### Storing the performance metrics in the dataframe

In [6]:
# Evaluate the model using the best parameters found through Grid Search CV 
c_matrix = confusion_matrix(y_test, grid_lregression.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model': "LR", 
                                                     'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                     'Precision': [TP/(TP+FP)], 
                                                     'Recall': [TP/(TP+FN)], 
                                                     'F1': [2*TP/(2*TP+FP+FN)]
                                                    }, index=[0])])

In [7]:
performance

,model,Accuracy,Precision,Recall,F1
0,LR,0.923077,0.938776,0.938776,0.938776


### Implementing SVM model with hyperparameters

##### Using Randomized search to find the best parameters

In [8]:
score_measure = "recall"
kfolds = 5

param_grid = {
    'C': np.arange(1,25),   
    'gamma': ['scale','auto'],
    'kernel':['linear','rbf','poly']
}

svm = SVC()
rand_search = RandomizedSearchCV(estimator = svm, param_distributions=param_grid, cv=kfolds, n_iter=140,
                           scoring=score_measure, verbose=1, n_jobs=-1, 
                           return_train_score=True)

_ = rand_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

Fitting 5 folds for each of 140 candidates, totalling 700 fits
The best recall score is 1.0
... with parameters: {'kernel': 'poly', 'gamma': 'scale', 'C': 1}


C:\Users\akash\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


##### Performing GridSearch over a close range of parameters that we got from Randomized search to find the best parameters

In [9]:
score_measure = "recall"
kfolds = 5

C = rand_search.best_params_['C']
gamma = rand_search.best_params_['gamma']
kernel = rand_search.best_params_['kernel']

param_grid = {
    'C': np.arange(C-2,C+2),  
    'gamma': [gamma],
    'kernel': [kernel]
    
}

svm1 = SVC()
grid_search = GridSearchCV(estimator = svm1, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1, # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestprecision_SVM = grid_search.best_estimator_

Fitting 5 folds for each of 4 candidates, totalling 20 fits
The best recall score is 1.0
... with parameters: {'C': 1, 'gamma': 'scale', 'kernel': 'poly'}


C:\Users\akash\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 20.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\akash\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\akash\Anaconda3\lib\site-packages\sklearn\svm\_base.py", line 255, in fit
    fit(X, y, sample_weight, solver_type, kernel, random_seed=seed)
  File "C:\Users\akash\Anaconda3\lib\site-packages\sklearn\svm\_base.py", line 315, in _dense_fit
    ) = libsvm.fit(
  File "sklearn\svm\_libsvm.pyx",

##### Storing the performance metrics in the dataframe

In [52]:
c_matrix = confusion_matrix(y_test, grid_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"SVM", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

NameError: name 'grid_search' is not defined

### Implementing Decision Tree model with hyperparameters

##### Using Randomized search to find the best parameters

In [11]:
score_measure = "recall"
kfolds = 5

param_grid = {
    'min_samples_split': np.arange(2,50),  
    'min_samples_leaf': np.arange(1,50),
    'min_impurity_decrease': np.arange(0.0001, 0.01, 0.0005),
    'max_leaf_nodes': np.arange(5, 50), 
    'max_depth': np.arange(1,20), 
    'criterion': ['gini', 'entropy'],
}

dtree = DecisionTreeClassifier()
rand_search = RandomizedSearchCV(estimator=dtree, param_distributions=param_grid, cv=kfolds, n_iter=500,
                                 scoring=score_measure, verbose=1, n_jobs=-1, # n_jobs=-1 will utilize all available CPUs 
                                 return_train_score=True)

_ = rand_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {rand_search.best_score_}")
print(f"... with parameters: {rand_search.best_params_}")

bestPrecTree = rand_search.best_estimator_

Fitting 5 folds for each of 500 candidates, totalling 2500 fits
The best recall score is 0.932020202020202
... with parameters: {'min_samples_split': 42, 'min_samples_leaf': 13, 'min_impurity_decrease': 0.0086, 'max_leaf_nodes': 19, 'max_depth': 13, 'criterion': 'entropy'}


##### Performing GridSearch over a close range of parameters that we got from Randomized search to find the best parameters

In [12]:
score_measure = "recall"
kfolds = 5

param_grid = {
    'min_samples_split': np.arange(26,36),  
    'min_samples_leaf': np.arange(8,16),
    'min_impurity_decrease': np.arange( 0.0005, 0.0010, 0.0020),
    'max_leaf_nodes': [10,30], 
    'max_depth': [5,15], 
    'criterion': ['entropy']
}


dtree = DecisionTreeClassifier()
grid_search = GridSearchCV(estimator = dtree, param_grid=param_grid, cv=kfolds, 
                           scoring=score_measure, verbose=1, n_jobs=-1,  # n_jobs=-1 will utilize all available CPUs 
                           return_train_score=True)

_ = grid_search.fit(X_train, y_train)

print(f"The best {score_measure} score is {grid_search.best_score_}")
print(f"... with parameters: {grid_search.best_params_}")

bestPrecisionTree = grid_search.best_estimator_

Fitting 5 folds for each of 320 candidates, totalling 1600 fits
The best recall score is 0.9094949494949495
... with parameters: {'criterion': 'entropy', 'max_depth': 5, 'max_leaf_nodes': 10, 'min_impurity_decrease': 0.0005, 'min_samples_leaf': 8, 'min_samples_split': 26}


##### Storing the performance metrics in the dataframe

In [13]:
c_matrix = confusion_matrix(y_test, grid_search.predict(X_test))
TP = c_matrix[1][1]
TN = c_matrix[0][0]
FP = c_matrix[0][1]
FN = c_matrix[1][0]
performance = pd.concat([performance, pd.DataFrame({'model':"Decision Tree", 
                                                    'Accuracy': [(TP+TN)/(TP+TN+FP+FN)], 
                                                    'Precision': [TP/(TP+FP)], 
                                                    'Recall': [TP/(TP+FN)], 
                                                    'F1': [2*TP/(2*TP+FP+FN)]
                                                     }, index=[0])])
performance

,model,Accuracy,Precision,Recall,F1
0,LR,0.923077,0.938776,0.938776,0.938776
0,SVM,0.628205,0.628205,1.000000,0.771654
0,Decision Tree,0.923077,0.930000,0.948980,0.939394


## Conclusion:
#### Looking at the performance metric dataframe we can see that the highest recall score of SVM model. Although it has the lowest accuracy, precision and F1 score.

#### Thus, it is evident that Decision tree model is the best model for predicting whether the water is safe or not as it has an recall score of 94 percent which is highest as compared to other metrics like accuracy, precision and F1.